In [1]:
#최종 코드 / 정확도 출력 X
#랜덤포레스트, LR, 결정트리, KNN 피싱 파일 넣은 것
import numpy as np
import pandas as pd
from tqdm import tqdm

#모델 생성
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
import joblib
warnings.filterwarnings(action='ignore')


In [2]:
#데이터 프레임으로 구성된 데이터 읽어오기
train_dataset = pd.read_csv('CVV_train1.csv')
data_column = train_dataset.columns

In [3]:
#훈련용 데이터셋 구성
print("Data Shape = ", np.shape(train_dataset))

X_data = train_dataset[data_column[1:-1]]
y_label = train_dataset[data_column[-1]]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.3)

Data Shape =  (10000, 12)


In [4]:
#개별 ML 모델을 위한 Classifier 생성
rf_clf = RandomForestClassifier(n_estimators = 230, random_state = 13, criterion = 'gini', max_depth = 100, max_leaf_nodes=350, bootstrap=False)
lr_clf = LogisticRegression(solver='newton-cg', random_state = 13)
dt_clf = DecisionTreeClassifier(max_depth = 12, random_state = 13)
knn_clf = KNeighborsClassifier(n_neighbors = 11)

#투표 분류
voting_clf = VotingClassifier(
   estimators = [('rf', rf_clf), ('lr', lr_clf), ('dt', dt_clf), ('knn', knn_clf)],
   voting='soft')

joblib.dump(voting_clf, './model.pkl')

voting_clf = voting_clf.fit(X_train, y_train)

#정확도 출력
print('정확도 : ', round(voting_clf.score(X_test, y_test)*100, 2))
    

정확도 :  93.53


In [5]:
#테스트 데이터셋
pred_file = pd.read_csv('CVV_tet1.csv')
data_column = pred_file.columns
pred_data = pred_file[data_column[1:]]

label_data = voting_clf.predict(pred_data)

print(label_data)

[1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1]


In [6]:
#url과 예측값을 csv파일로 출력
pred_file['label']=label_data
result_dataset = pred_file[['url','label']]
result_dataset.to_csv('CVV_result1.csv', index=False)

print("Finished")

Finished
